In [1]:
import os
from pdf2image import convert_from_path
import cv2
import PIL
import pytesseract
from pytesseract import Output
import pandas as pd
import time
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_samples, silhouette_score
import math
import csv

In [2]:
def get_string(img_path):
    # Read image using opencv
    img = cv2.imread(img_path)

    # Extract the file name without the file extension
    file_name = os.path.basename(img_path).split('.')[0]
    file_name = file_name.split()[0]

    # Create a directory for outputs
    output_path = os.path.join(output_dir, file_name)
    if not os.path.exists(output_path):
        os.makedirs(output_path)

box=""
pdf_dir = r"/Users/KP/Documents/"
os.chdir(pdf_dir)

for pdf_file in os.listdir(pdf_dir):
    if pdf_file.endswith(".pdf"):
        pages = convert_from_path(pdf_file, 300)
        pdf_file = pdf_file[:-4]
        for page in pages:
            page.save("%s-page%d.jpg" % (pdf_file,pages.index(page)), "JPEG", dpi=(300,300))
            img = cv2.imread(pdf_file+'-page'+str(pages.index(page))+'.jpg')
            d = pytesseract.image_to_data(img, output_type=Output.DICT)
            n_boxes = len(d['level'])
            paragraph_boxes=[]
            for i in range(n_boxes):
                (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
                if d['level'][i]==2:#d['level'][i]!=1 and d['level'][i]!=2 and d['level']!=3:
                    a=str(x)+"\t"+str(y)+"\t"+str(w)+"\t"+str(h)
                    paragraph_boxes.append(a)
                    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    #print(i,d['level'][i],"\t",d['left'][i], d['top'][i], d['width'][i], d['height'][i])
            cv2.imwrite(pdf_file+'-page'+str(pages.index(page))+'_bounded.jpg', img)
            ctr=0
            list_words=[]
            temp_list=[]
            flag=0
            start=0
            end=0
            output_file=open(pdf_file+'-page'+str(pages.index(page))+".csv",'w')
            box="X1,Y1,X2,Y2,Label\n"
            for i in range(n_boxes):
                (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
                if d['level'][i]==5:#d['level'][i]!=1 and d['level'][i]!=2 and d['level']!=3:
                    para_x1=int(paragraph_boxes[ctr].split("\t")[0])
                    para_y1=int(paragraph_boxes[ctr].split("\t")[1])
                    para_x2=int(int(paragraph_boxes[ctr].split("\t")[0])+int(paragraph_boxes[ctr].split("\t")[2]))
                    para_y2=int(int(paragraph_boxes[ctr].split("\t")[1])+int(paragraph_boxes[ctr].split("\t")[3]))
                    word_x2=int(int(a.split("\t")[0])+int(a.split("\t")[2]))
                    word_y2=int(int(a.split("\t")[1])+int(a.split("\t")[3]))
                    if (d['left'][i]>=para_x1) & (d['top'][i]>=para_y1):
                        #print(ctr,paragraph_boxes[ctr],para_x2,para_y2)
                        #print(i,d['left'][i], d['top'][i],word_x2,word_y2)
                        if(word_x2>para_x2)&(word_y2>para_y2):
                            end=i
                            for j in range(start,end):
                                if d['level'][j]==5:
                                    x1y1=(d['left'][j],d['top'][j])
                                    x2y2=(word_x2,word_y2)
                                    word=(x1y1,x2y2)
                                    #print(word)
                                    temp_list.append(word)
                                    #print(temp_list)
                                    flag=1
                            if flag==1:
                                ctr+=1
                                start=end
                                list_words.append(temp_list)
                                box+=str(para_x1)+","+str(para_y1)+","+str(para_x2)+","+str(para_y2)+","+"Paragraph\n"
                                print(box)
                                temp_list=[]
                                flag=0
            output_file.write(box)
            output_file.close()

<h1> Creating CSV for words within table

In [ ]:
box=""
pdf_dir = r"/Users/KP/Documents/"
os.chdir(pdf_dir)
for pdf_file in os.listdir(pdf_dir):
    if pdf_file.endswith(".pdf"):
        pages = convert_from_path(pdf_file, 300)
        pdf_file = pdf_file[:-4]
        for page in pages:
            page.save("%s-page%d.jpg" % (pdf_file,pages.index(page)), "JPEG", dpi=(300,300))
            img = cv2.imread(pdf_file+'-page'+str(pages.index(page))+'.jpg')
            d = pytesseract.image_to_data(img, output_type=Output.DICT)
            n_boxes = len(d['level'])
            #cv2.imwrite(pdf_file+'-page'+str(pages.index(page))+'_bounded.jpg', img)
            ctr=0
            list_words=[]
            temp_list=[]
            flag=0
            start=0
            end=0
            output_file=open(pdf_file+'-page'+str(pages.index(page))+".csv",'w')
            box="X1,Y1,X2,Y2,Label,Word\n"
            if pages.index(page)==2:
                for i in range(n_boxes):
                    (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
                    if d['level'][i]==5:#d['level'][i]!=1 and d['level'][i]!=2 and d['level']!=3:
                        word_x1=d['left'][i]
                        word_y1=d['top'][i]
                        word_x2=d['left'][i]+d['width'][i]
                        word_y2=d['top'][i]+d['height'][i]
                        if ((word_x1>=240 and word_y1>=693) and (word_x2<=2149 and word_y2<=1345)):
                            print(d['text'][i])
                            box+=str(word_x1)+","+str(word_y1)+","+str(word_x2)+","+str(word_y2)+","+"Table Cell,"+d['text'][i]+"\n"
                output_file.write(box)
                output_file.close()

<h1> Reading the created CSV and generate word bounding boxes within table

In [ ]:
df=pd.read_csv("c4611_sample_explain-page2.csv",header=0)
li=[]
Xlist=[]
Ylist=[]
coord=[]
l=[]

In [ ]:
img = cv2.imread('c4611_sample_explain-page2.jpg')
for i in range(len(df['X1'])):
    (x, y, w, h) = (df['X1'][i], df['Y1'][i], df['X2'][i]-df['X1'][i], df['Y2'][i]-df['Y1'][i])
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    temp=(df['X1'][i],df['X2'][i])#+df['Y2'][i])/2)
    Xlist.append((df['X1'][i]+df['X2'][i])/2)
    Ylist.append((df['Y1'][i]+df['Y2'][i])/2)
    coord.append([Xlist[i],Ylist[i]])
    if (i<len(df['Y1'])-1) and ((df['Y1'][i+1]-df['Y1'][i])/df['Y1'][i])<0.05:
        print(df['Y1'][i],df['Y1'][i+1])
        l.append(df['X1'][i])
        print(df['Word'][i],df['Word'][i+1])
    #temp=(df['X1'][i],df['Y1'][i])
    li.append(temp)
    cv2.imwrite('Custom_bounded.jpg', img)

<h1> Sort the words to identify text in same row

In [ ]:
import math
ctr=0
l=[]
i=0
Final=[]
while(i<len(df)):
    ctr=i
    while(ctr<len(df['X1'])):
        if (ctr<len(df['Y1'])-1) and ((df['Y1'][ctr+1]-df['Y1'][ctr])/df['Y1'][ctr])<0.05:
            #print(df['Y1'][ctr],df['Y1'][ctr+1],math.sqrt(abs(coord[ctr+1][1]**2)-(coord[ctr][1]**2)),df['Word'][ctr])
            print(df['Word'][ctr],ctr,df['Word'][ctr+1])
            l.append([float(df['X1'][ctr]),float(df['X2'][ctr]),float(math.sqrt(abs((coord[ctr+1][1]**2)-(coord[ctr][1]**2)))),df['Word'][ctr],ctr])
            ctr+=1
        else:
            if (ctr<len(df['Y1'])-1) and ((df['Y1'][ctr]-df['Y1'][ctr-1])/df['Y1'][ctr-1])<0.05:
                print(df['Word'][ctr],ctr,df['Word'][ctr+1])
                l.append([float(df['X1'][ctr]),float(df['X2'][ctr]),float(math.sqrt(abs((coord[ctr+1][1]**2)-(coord[ctr][1]**2)))),df['Word'][ctr],ctr])
                ctr+=1
                i=ctr
            else:
                ctr+=1
                i=ctr
            if (len(l)>0):
                Final.append(l)
            l=[]
            break

In [ ]:
X=[]
X1=[]
for item in Final:
    for i in item:
        X.append(i[:3])
        X1.append(i)
X2=np.array(X)

<h1> Use K-means clustering to identify columns within the table

In [ ]:
wcss = []

for i in range(1, len(X2)):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
    kmeans.fit(X2)
    wcss.append(kmeans.inertia_)
    
#Plotting the results onto a line graph, allowing us to observe 'The elbow'
plt.plot(range(1, len(X2)), wcss)
plt.title('The elbow method')
plt.xlabel('Number of clusters')
plt.xlim(1,10)
plt.ylabel('WCSS') #within cluster sum of squares
plt.show()

<h2>Identifying Optimum Clusters using Silhouette Score

In [ ]:
maximum=-9999999
optimum_cluster=0
for n_clusters in range(2,len(X2)):
    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X2)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X2, cluster_labels)
    if silhouette_avg>maximum:
        maximum=silhouette_avg
        optimum_cluster=n_clusters

In [ ]:
print("Optimum Clusters identified at:",optimum_cluster)

In [ ]:
kmeans = KMeans(n_clusters = optimum_cluster, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
y_kmeans = kmeans.fit_predict(X2)

In [ ]:
for i in range(len(X1)):
    print(X1[i],"Cluster",y_kmeans[i])

<h1> TCD

In [ ]:
import math
ctr=0
l=[]
col={}
i=0
for i in range(len(y_kmeans)):
    col[y_kmeans[i]]=[]
for key in col.keys():
    temp=[]
    for i in range(len(y_kmeans)):
        if y_kmeans[i]==key:
            temp.append(X1[i][4])            
    col[key]=temp

In [ ]:
col

In [ ]:
cell=[]
for key in col:
    temp=[]
    for i in range(len(col[key])):
        #print(df['Word'][col[key][i]],i)
        if i!=len(col[key])-1:
            if ((abs(df['Y1'][col[key][i]]-df['Y1'][col[key][i+1]]))/df['Y1'][col[key][i]])<0.05:
                print(df['Word'][col[key][i]],"Cell")
                temp.append([df['Word'][col[key][i]],df['X1'][col[key][i]],
                             df['Y1'][col[key][i]],df['X2'][col[key][i]],df['Y2'][col[key][i]]])
            else:
                temp.append([df['Word'][col[key][i]],df['X1'][col[key][i]],
                             df['Y1'][col[key][i]],df['X2'][col[key][i]],df['Y2'][col[key][i]]])
                cell.append(temp)
                temp=[]
        else:
            if df['Y1'][col[key][i]]-df['Y1'][col[key][i-1]]<0.05:
                print(df['Word'][col[key][i]],"Cell")
                temp.append([df['Word'][col[key][i]],df['X1'][col[key][i]],
                             df['Y1'][col[key][i]],df['X2'][col[key][i]],df['Y2'][col[key][i]]])
                cell.append(temp)
                temp=[]
            else:
                temp.append([df['Word'][col[key][i]],df['X1'][col[key][i]],
                             df['Y1'][col[key][i]],df['X2'][col[key][i]],df['Y2'][col[key][i]]])
                cell.append(temp)
                temp=[]

<h1>Checking functionality on training data

In [3]:
box=""
img_dir = r"/Users/KP/Documents/TCD/Data/"
os.chdir(img_dir)
image_frame=pd.read_csv("train.csv")
img_dir = r"/Users/KP/Documents/TCD/Data/images/"
os.chdir(img_dir)

In [30]:
box=""
img_dir = r"/Users/KP/Documents/TCD/Data/Test"
os.chdir(img_dir)
image_frame=pd.read_csv("UniQ_Clustered.csv",header=0)

In [31]:
count=0
for image_file in os.listdir(img_dir):
    if image_file.endswith(".png"):
        for i in range(len(image_frame['image_id'])):
            if (image_file==image_frame['image_id'][i]):
                count+=1
                print(image_file,image_frame['image_id'][i],count)
                img = cv2.imread(image_file)
                d = pytesseract.image_to_data(img, output_type=Output.DICT)
                n_boxes = len(d['level'])
                #cv2.imwrite(image_file+'_bounded.jpg', img)
                ctr=0
                list_words=[]
                temp_list=[]
                flag=0
                start=0
                end=0
                output_file=open(image_file[:-4]+".csv",'w')
                box="X1,Y1,X2,Y2,Label,Word\n"
                for j in range(n_boxes):
                    (x, y, w, h) = (d['left'][j], d['top'][j], d['width'][j], d['height'][j])
                    if d['level'][j]==5:#d['level'][i]!=1 and d['level'][i]!=2 and d['level']!=3:
                        word_x1=d['left'][j]
                        word_y1=d['top'][j]
                        word_x2=d['left'][j]+d['width'][j]
                        word_y2=d['top'][j]+d['height'][j]
                        if ((word_x1>=image_frame['xmin'][i] and word_y1>=image_frame['ymin'][i]) and (word_x2<=image_frame['xmax'][i] and word_y2<=image_frame['ymax'][i])):
                            #print(d['text'][i])
                            box+=str(word_x1)+","+str(word_y1)+","+str(word_x2)+","+str(word_y2)+","+"Table Cell,"+d['text'][j]+"\n"
                output_file.write(box)
                output_file.close()
            else:
                continue

In [21]:
def image_create(filename,df):
    li=[]
    Xlist=[]
    Ylist=[]
    coord=[]
    l=[]
    img = cv2.imread(filename[:-4]+".png")
    print("Image:"+filename+".png being created")
    for i in range(len(df['X1'])):
        (x, y, w, h) = (df['X1'][i], df['Y1'][i], df['X2'][i]-df['X1'][i], df['Y2'][i]-df['Y1'][i])
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        temp=(df['X1'][i],df['X2'][i])#+df['Y2'][i])/2)
        Xlist.append((df['X1'][i]+df['X2'][i])/2)
        Ylist.append((df['Y1'][i]+df['Y2'][i])/2)
        coord.append([Xlist[i],Ylist[i]])
        if (i<len(df['Y1'])-1) and ((df['Y1'][i+1]-df['Y1'][i])/df['Y1'][i])<0.05:
            print(df['Y1'][i],df['Y1'][i+1])
            l.append(df['X1'][i])
            print(df['Word'][i],df['Word'][i+1])
            #temp=(df['X1'][i],df['Y1'][i])
        li.append(temp)
        cv2.imwrite(filename[:-4]+'custom.jpg', img)
    #print(coord)
    return coord

In [22]:
def row_id(df,coord):
    ctr=0
    l=[]
    i=0
    Final=[]
    while(i<len(df['X1'])):
        ctr=i
        while(ctr<len(df['X1'])):
            if (ctr<len(df['Y1'])-1) and ((df['Y1'][ctr+1]-df['Y1'][ctr])/df['Y1'][ctr])<0.05:
                #print(df['Y1'][ctr],df['Y1'][ctr+1],math.sqrt(abs(coord[ctr+1][1]**2)-(coord[ctr][1]**2)),df['Word'][ctr])
                print(df['Word'][ctr],ctr,df['Word'][ctr+1])
                l.append([float(df['X1'][ctr]),float(df['X2'][ctr]),float(math.sqrt(abs((coord[ctr+1][1]**2)-(coord[ctr][1]**2)))),df['Word'][ctr],ctr])
                ctr+=1
            else:
                if (ctr<len(df['Y1'])-1) and ((df['Y1'][ctr]-df['Y1'][ctr-1])/df['Y1'][ctr-1])<0.05:
                    print(df['Word'][ctr],ctr,df['Word'][ctr+1])
                    l.append([float(df['X1'][ctr]),float(df['X2'][ctr]),float(math.sqrt(abs((coord[ctr+1][1]**2)-(coord[ctr][1]**2)))),df['Word'][ctr],ctr])
                    ctr+=1
                    i=ctr
                else:
                    ctr+=1
                    i=ctr
                if (len(l)>0):
                    Final.append(l)
                l=[]
                break
    X=[]
    X1=[]
    for item in Final:
        for i in item:
            X.append(i[:3])
            X1.append(i)
    X2=np.array(X)
    return (X1,X2)

In [23]:
def K_Means(X2):
    wcss = []

    for i in range(1, len(X2)):
        kmeans = KMeans(n_clusters = i, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
        kmeans.fit(X2)
        wcss.append(kmeans.inertia_)

    #Plotting the results onto a line graph, allowing us to observe 'The elbow'
    plt.plot(range(1, len(X2)), wcss)
    plt.title('The elbow method')
    plt.xlabel('Number of clusters')
    plt.xlim(1,10)
    plt.ylabel('WCSS') #within cluster sum of squares
    plt.show()
    maximum=-9999999
    optimum_cluster=0
    for n_clusters in range(2,len(X2)):
        # Initialize the clusterer with n_clusters value and a random generator
        # seed of 10 for reproducibility.
        clusterer = KMeans(n_clusters=n_clusters, random_state=10)
        cluster_labels = clusterer.fit_predict(X2)

        # The silhouette_score gives the average value for all the samples.
        # This gives a perspective into the density and separation of the formed
        # clusters
        silhouette_avg = silhouette_score(X2, cluster_labels)
        if silhouette_avg>maximum:
            maximum=silhouette_avg
            optimum_cluster=n_clusters
    print("Optimum Clusters identified at:",optimum_cluster)
    kmeans = KMeans(n_clusters = optimum_cluster, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
    y_kmeans = kmeans.fit_predict(X2)
    return (optimum_cluster,y_kmeans)

In [24]:
def TCD(X1,y_kmeans):
    ctr=0
    l=[]
    col={}
    i=0
    for i in range(len(y_kmeans)):
        col[y_kmeans[i]]=[]
    for key in col.keys():
        temp=[]
        for i in range(len(y_kmeans)):
            if y_kmeans[i]==key:
                temp.append(X1[i][4])            
        col[key]=temp
    cell=[]
    for key in col:
        temp=[]
        for i in range(len(col[key])):
            #print(df['Word'][col[key][i]],i)
            if i!=len(col[key])-1:
                if ((abs(df['Y1'][col[key][i]]-df['Y1'][col[key][i+1]]))/df['Y1'][col[key][i]])<0.05:
                    print(df['Word'][col[key][i]],"Cell")
                    temp.append([df['Word'][col[key][i]],df['X1'][col[key][i]],
                                 df['Y1'][col[key][i]],df['X2'][col[key][i]],df['Y2'][col[key][i]]])
                else:
                    temp.append([df['Word'][col[key][i]],df['X1'][col[key][i]],
                                 df['Y1'][col[key][i]],df['X2'][col[key][i]],df['Y2'][col[key][i]]])
                    cell.append(temp)
                    temp=[]
            else:
                if df['Y1'][col[key][i]]-df['Y1'][col[key][i-1]]<0.05:
                    print(df['Word'][col[key][i]],"Cell")
                    temp.append([df['Word'][col[key][i]],df['X1'][col[key][i]],
                                 df['Y1'][col[key][i]],df['X2'][col[key][i]],df['Y2'][col[key][i]]])
                    cell.append(temp)
                    temp=[]
                else:
                    temp.append([df['Word'][col[key][i]],df['X1'][col[key][i]],
                                 df['Y1'][col[key][i]],df['X2'][col[key][i]],df['Y2'][col[key][i]]])
                    cell.append(temp)
                    temp=[]
    return cell

In [25]:
def clustered_image(filename,cells):
    img = cv2.imread(filename[:-4]+".png")
    for cell in cells:
        xstart=9999
        ystart=9999
        xend=-9999
        yend=-9999
        for word in cell:
            if word[1]<xstart:
                xstart=word[1]
            if word[2]<ystart:
                ystart=word[2]
            if word[3]>xend:
                xend=word[3]
            if word[4]>yend:
                yend=word[4]
        (x, y, w, h) = (xstart,ystart,xend-xstart,yend-ystart)
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.imwrite(filename[:-4]+"_Clustered.png", img)

In [26]:
def Clustered_File(csv_file,cells):
    string="Word,X1,Y1,X2,Y2,Cell,Number of clusters\n"
    count=0
    for cell in cells:
        count+=1
        for subitem in cell:
            string+=str(subitem[0])+","+str(subitem[1])+","+str(subitem[2])+","+str(subitem[3])+","+str(subitem[4])+","+str(count)+","+str(n_cluster)+"\n"
    Output_File=open("Clustered_"+csv_file,"w")
    Output_File.write(string)
    Output_File.close()

In [29]:
flag=0
for csv_file in os.listdir(img_dir):
    if csv_file.endswith(".csv"):
        print(csv_file)
        try:
            df=pd.read_csv(csv_file,usecols=[0,1,2,3,4,5],header=0)
            coord=image_create(csv_file,df)
            (X1,X2)=row_id(df,coord)
            (n_cluster,y_kmeans)=K_Means(X2)
            cells=TCD(X1,y_kmeans)
            clustered_image(csv_file,cells)
            Clustered_File(csv_file,cells)
            flag=1
        except (KeyError,IndexError):
            pass

UQ_Clustered.csv
Image:UQ_Clustered.csv.png being created
1533 575
UniQ_Clustered.csv
Image:UniQ_Clustered.csv.png being created
1583 628
Clustered.csv
Image:Clustered.csv.png being created
2264 613
